文本张量表示：
- 将一段文本使用张量进行表示，其中一般将词汇表示成向量，称作词向量。再由各个词向量按顺序组成矩阵形成文本表示。
- 作用：能够使语言文本作为计算机处理程序的输入，进行接下来一系列的解析工作。

one-hot词向量表示：
- 独热编码，将每个词表示成具有$n$个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0的位置不同，其中$n$的大小是整个语料中不同词汇的总数。
- 优点：操作简单，容易理解
- 劣势：完全割裂了词与词之间的联系。且在大语料集下，每个向量的长度过大，占据大量内存。

In [4]:
# from sklearn.externals import joblib 替换为
import joblib

In [5]:
from keras.preprocessing.text import Tokenizer

In [7]:
vocab = {'周杰伦', '陈奕迅', '王力宏', '李宗盛', '吴亦凡', '鹿晗'}

t = Tokenizer(num_words=None,char_level=False)
t.fit_on_texts(vocab)

for token in vocab:
    zero_list = [0] * len(vocab)
    # 使用映射器转化现有文本数据，返回样式[[2]]
    token_index = t.texts_to_sequences([token])[0][0] - 1
    zero_list[token_index] = 1
    print(token, "one-hot编码为：", zero_list)
   
# 保存映射器
tokenizer_path = './Tokenizer'
joblib.dump(t, tokenizer_path)

鹿晗 one-hot编码为： [1, 0, 0, 0, 0, 0]
陈奕迅 one-hot编码为： [0, 1, 0, 0, 0, 0]
吴亦凡 one-hot编码为： [0, 0, 1, 0, 0, 0]
李宗盛 one-hot编码为： [0, 0, 0, 1, 0, 0]
王力宏 one-hot编码为： [0, 0, 0, 0, 1, 0]
周杰伦 one-hot编码为： [0, 0, 0, 0, 0, 1]


['./Tokenizer']

In [11]:
# 加载保存的映射器
t = joblib.load(tokenizer_path)

In [12]:
token = '李宗盛'
token_index = t.texts_to_sequences([token])[0][0] - 1
zero_list = [0] * len(vocab)
zero_list[token_index] = 1
print(token, "one-hot编码为：", zero_list)

李宗盛 one-hot编码为： [0, 0, 0, 1, 0, 0]


word2vec：
- 将词汇表示成向量的无监督训练方法，包含CBOW和skipgram两种训练模式。

CBOW（Continuous bag of words）模式：
- 给定一段用于训练的文本语料，再选定某段长度（窗口）作为研究对象，使用上下文词汇预测目标词汇。
- 如窗口大小为9，使用前后4个词汇对目标词汇进行预测。

skipgram模式：
- 选定某段窗口作为研究对象，使用目标词汇预测上下文词汇。



使用fasttext工具实现word2vec的训练和使用：
- 获取训练数据
- 训练词向量
- 模型超参数设定
- 模型效果检验
- 模型的保存与重加载

- http://www.mattmahoney.net/dc/enwik9.zip 数据

perl wikifil.pl data/enwik9 > data/fil9

head -c 80 data/fil9

In [1]:
import fasttext

In [2]:
# 使用无监督训练方法进行词向量的训练
model = fasttext.train_unsupervised('data/WutheringHeights.txt', model='skipgram')
# model = fasttext.train_unsupervised('./data/fil9.txt', model='skipgram', dim=300, epoch=1, lr=0.1, thread=8)

In [7]:
# 获得指定词汇的词向量
model.get_word_vector('the')

array([-0.02344137,  0.05361558, -0.03799271,  0.03629256, -0.03852846,
        0.25951955,  0.02942149, -0.1479783 ,  0.13530746,  0.00449662,
       -0.00403729,  0.10772362, -0.13828377,  0.00563046, -0.03574153,
       -0.14859238, -0.04394291, -0.1392908 , -0.04615721,  0.10498832,
        0.2888667 ,  0.3572071 , -0.05442623, -0.01993471,  0.09003352,
        0.22741356,  0.01993065, -0.06570473, -0.2111397 , -0.10435101,
        0.00988963, -0.05863618, -0.00388045,  0.14565462, -0.42046502,
        0.15249185, -0.03636006,  0.31842825,  0.17958552,  0.26985833,
        0.01489444,  0.0682444 , -0.12343358,  0.01709484,  0.1945873 ,
       -0.27542168,  0.04431907,  0.05998012, -0.01459279, -0.19819492,
       -0.16854905,  0.0511236 , -0.11347513, -0.0280004 ,  0.02549928,
        0.09817851, -0.16742094, -0.05544618,  0.14811103, -0.10091434,
        0.17067082, -0.11870677,  0.16837458, -0.00892791, -0.03642026,
       -0.02412478, -0.06913874, -0.03687835, -0.11603368, -0.26

In [11]:
# 无监督训练模式：默认为'skipgram'，或者'cbow'
# 词嵌入维度dim：默认为100，但随着语料库的增大，词嵌入的维度往往也要更大
# 数据循环次数epoch：默认为5
# 学习率lr：默认为0.05，建议选择[0.01, 1]范围内
# 使用的线程数thread：默认为12个线程，一般建议和cpu核数相同
model = fasttext.train_unsupervised('./data/fil9.txt', model='cbow', dim=300, epoch=1, lr=0.1, thread=8)

# 训练的词数量有问题

检查单词向量质量的一种简单方法就是查看其邻近单词，通过主观来判断这些邻近单词是否与目标单词相关来粗略评定模型效果的好坏


In [3]:
# 查看'sports'的邻近单词
model.get_nearest_neighbors('music')

[(0.9995617270469666, 'anything,'),
 (0.9995493292808533, 'amuse'),
 (0.9995458126068115, 'nothing,'),
 (0.9995400309562683, 'cannot'),
 (0.9995284080505371, "me.'"),
 (0.9995243549346924, 'home,'),
 (0.9995202422142029, 'creature'),
 (0.9995143413543701, 'excuse'),
 (0.9995019435882568, 'sorry'),
 (0.999500036239624, "it!'")]

In [3]:
f = open('./data/fil9.txt', 'r', encoding='utf-16')
print(f.read(80))

 anarchism originated as a term of abuse first used against early working class 


In [4]:
model.save_model('fil9.bin')

In [5]:
model = fasttext.load_model('fil9.bin')

In [6]:
model.get_word_vector('the')

array([-4.63873893e-02,  6.95816875e-02, -5.18368818e-02,  5.52614853e-02,
       -6.63495958e-02,  2.60103375e-01,  2.88014021e-02, -1.35873348e-01,
        1.21366061e-01,  1.84253678e-02, -2.13505644e-02,  1.10273302e-01,
       -1.67804420e-01,  5.33773098e-03, -3.13052498e-02, -1.63602635e-01,
       -3.71532030e-02, -1.15687028e-01, -4.87629399e-02,  1.35450810e-01,
        2.85685867e-01,  3.95034045e-01, -4.71343957e-02, -3.66585366e-02,
        6.52566850e-02,  1.86495647e-01,  2.44210474e-03, -7.00538903e-02,
       -2.13160127e-01, -1.28166139e-01, -4.73677414e-04, -8.64003375e-02,
       -5.19052660e-03,  1.28203973e-01, -4.11855578e-01,  1.63670182e-01,
       -2.18197070e-02,  3.24532032e-01,  1.28520995e-01,  2.51120090e-01,
       -3.09819647e-04,  4.29519340e-02, -1.59247592e-01,  2.84081753e-02,
        1.66341379e-01, -2.51592815e-01,  4.07395475e-02,  6.46741539e-02,
       -2.66525112e-02, -1.69886410e-01, -1.04285292e-01,  8.37027840e-03,
       -1.00598529e-01, -

word embedding（词嵌入）：
- 广义的word embedding包括所有密集词汇向量的表示方法，如word2vec
- 狭义的word embedding指在神经网络中加入的embedding层，对整个网络进行训练的同时产生的embedding矩阵。

In [1]:
import fileinput
import torch 
import json 
from torch.utils.tensorboard import SummaryWriter

In [2]:
import pandas as pd

df = pd.read_csv('./data/vocab100.csv', header=None)

In [3]:
df['word'] = df[0].apply(lambda x: x.strip())

In [4]:
# 防止gfile包冲突
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

meta = df['word'].tolist()

writer = SummaryWriter()
# 代表100个词汇，每个词汇被表示成50维的向量
embedded = torch.randn(100, 50)
writer.add_embedding(embedded, metadata=meta)
writer.close()

In [ ]:
# tensorboard --logdir runs --host 0.0.0.0 
# http://localhost:6006/#projector